In [9]:
import os
import json
import pandas as pd
import folium

# Define a class to represent a vessel
class Vessel:
    def __init__(self, imo, lat, lon, speed, course):
        self.imo = imo
        self.lat = lat
        self.lon = lon
        self.speed = speed
        self.course = course

# Function to read AIS JSON files from a folder and convert them into a list of Vessel objects
def read_ais_json_files(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    # Filter only JSON files
    json_files = [file for file in files if file.endswith('.json')]

    # List to store vessel objects
    vessels = []

    # Read each JSON file and extract vessel information
    for file in json_files:
        with open(os.path.join(folder_path, file), 'r') as f:
            data = json.load(f)
            for entry in data['data']:
                # Extract relevant data (assuming JSON structure)
                vessel_imo = entry['vessel']['imo']
                lat = entry['navigation']['location']['lat']
                lon = entry['navigation']['location']['long']
                speed = entry['navigation']['speed']
                course = entry['navigation']['course']
                # Create Vessel object and append to list
                vessel = Vessel(vessel_imo, lat, lon, speed, course)
                vessels.append(vessel)

    return vessels

# Function to create a map using folium for a specific vessel
def create_map_for_vessel(vessels, target_vessel_id):
    # Filter vessels to include only the target vessel
    target_vessel = [vessel for vessel in vessels if vessel.imo == target_vessel_id]

    if not target_vessel:
        print(f"Vessel with ID {target_vessel_id} not found.")
        return None

    target_vessel = target_vessel[0]  # Extract the first (and only) element

    # Initialize map centered at the target vessel's location
    map_center = [target_vessel.lat, target_vessel.lon]
    m = folium.Map(location=map_center, zoom_start=10)

    # Add marker for the target vessel
    popup = f"Vessel IMO: {target_vessel.imo}<br>Speed: {target_vessel.speed}<br>Course: {target_vessel.course}"
    folium.Marker(location=[target_vessel.lat, target_vessel.lon], popup=popup).add_to(m)

    return m

# Example usage:
folder_path = '.\Data'  # Update this with the path to your AIS JSON files folder
vessels = read_ais_json_files(folder_path)
target_vessel_id = 9516650  # Specify the IMO of the vessel you want to display
map = create_map_for_vessel(vessels, target_vessel_id)

if map:
    map.save('test.html')  # Save the map as an HTML file